In [45]:
def getchar(m):
    """returns the next character from the input string m"""
    global p
    p+=1
    return m[p] if p < len(m) else ""

def isdigit(c):
    """returns True if input character c is a digit"""
    Digits = [str(x) for x in range(10)]
    return True if c in Digits else False

def isalpha(c):
    """returns True if input character c is a letter"""
    Uppers = [chr(x) for x in range(ord('A'),ord('Z')+1)]
    Lowers = [chr(x) for x in range(ord('a'),ord('z')+1)]
    return True if c in Uppers or c in Lowers or c == '_' else False

def isalnum(c):
    """returns True if the input character c is a digit or alphabetic"""
    return isdigit(c) or isalpha(c)

def isopFirst(c):
    """return true if input character is the first character of a op or relop"""
    first = ['+', '-', '*', '/', '=', '%', '&', '|', '^', '>', '<', '~', '!', '?']
    return True if c in first else False

def ispun(c):
    """returns True if the input character c is a defined punctuation"""
    puns = ['(',')',';','{','}','[',']', ':', ',']
    return True if c in puns else False

In [46]:
def init():
    """initialize the symbol table for lexical analysis"""
    global p                 # current input position
    p = -1

    global line_num          # current line number
    line_num = 1
    
    global ST                # symbol table
    ST = {}                  # Symbol Table for variables
    ST.setdefault('', 0)
    
    ST["int"] = ("int", "INT")
    ST["float"] = ("float", "FLOAT")
    ST["if"] = ("if","IF")             # adding in reserved words
    ST["else"] = ("else","ELSE")
    ST["for"] = ("for","FOR")
    ST["while"] = ("while","WHILE")
    ST["do"] = ("do","DO")
    ST["break"] = ("break", "BREAK")
    ST["continue"] = ("continue", "CONTINUE")
    ST["goto"] = ("goto", "GOTO")
    ST["label"] = ("label", "LABEL")
    
    global tokenval          # Value for the current token
    global DONE              # Flag value for end of iteration
    DONE = object()
    
    global lookahead         # lookahead token/symbol
    lookahead = None
    
    global err_flag
    err_flag = False

In [47]:
def lexan(m):
    """modified lexical analyzer from section 2.9, page 74"""
    global p           # string position
    global line_num    # line number
    global ST          # symbol table
    global tokenval    # value of current token
    global DONE        # flag for end of file
    global err_flag
    
    tokenval = None    # default value for tokens
    lexeme = ""        # current lexeme being read
    
    while True:
        c = getchar(m)                               # read first character
        
        if c in [' ','\t']:                          # skip over space and tab
            pass
        elif c == '\n':
            line_num += 1                            # increment line counter if newline is read
        
        elif isdigit(c):
            tokenval = int(c)                        # start with integer value if char is a digit
            c = getchar(m)                           # load next character
            while isdigit(c):
                tokenval = 10*tokenval + int(c)      # process next integer if char is a digit
                c = getchar(m)                       # load next character
            p -= 1
            return "NUM"
        
        elif isalpha(c):
            lexeme += c                              # append character to buffer
            c = getchar(m)                           # load next character
            while isalnum(c):
                lexeme += c                          # append character to buffer
                c = getchar(m)                       # load next character
            if lexeme not in ST.keys():
                ST[lexeme] = (lexeme, "ID")          # add entry to symbol table (if not already present)
            tokenval = lexeme
            p-=1
            return ST[lexeme][1]                     # return the token for the identifier lexeme
        
        elif p >= len(m): return DONE                # return DONE Flag for end of iteration
        
        elif isopFirst(c):
            douRelop = ['<=', '=<', '>=','=>', '==', '!=']
            douOp = ['=+', '+=', '-=', '=-', '*=', '=*', '/=', '=/', '++', '--', '>>', '<<']
            ops = ['+','-','*','/','=', '%', '&', '|', '^', '~', '!', '?']
            rops = ['<','>']
            comm = ['//']
            
            tokenval = c #append to tokenval and checkop
            checkOP =  c
            
            d = getchar(m) # get next token
            checkOP = checkOP + d #append to check for two digit operators
            
            
            if checkOP in douRelop: #check for two character relops
                tokenval=checkOP
                return "RELOP"
            elif checkOP in douOp: #check for two character ops
                tokenval = checkOP
                return "OP"
            elif checkOP in comm:
                while '\n' not in checkOP:
                    c = getchar(m)
                    checkOP += c #append to check for two digit operators
                line_num += 1

                token_val = checkOP
                return "COMMENT"
            else:
                if c in ops: #return single character op
                    p -= 1
                    return "OP"
                elif c in rops: #return single character relop
                    p -= 1
                    return "RELOP"
               
        elif ispun(c):
            tokenval = c
            return "PUN"                             # token is a operator +,-,*, or /
        
        else:
            print("invalid character",c,"found on line",line_num)

    

In [49]:
%run CompilerTheory_Ch5(Tree_Classes).ipynb

In [2]:
def parse(m):
    #production code -> stmt code | e
    global v
    global lookahead
    global tokenval
    global label_list
    global S
    global err_flag
    tree1 = MutableLinkedTree()
    p = tree1.add_root("code")
    
    if lookahead == None: lookahead = lexan(m)  # read first token
    if v: print("code: lookahead =",lookahead,", tokenval =",tokenval)
    
    while lookahead != DONE and tokenval != '}': #while code is not finished, find stmt
        
        child1 = tree1.add_child(p, "stmt")
        p = tree1.add_child(p, 'code')
        
        stmt(m)
        
        if vv:                                         #-------DEBUG TRANSLATION---------
            print("size of stack:",len(S))
            for T in S:  printTree(T, 0.5, 2000, 14)   #-------DEBUG TRANSLATION---------
        
        subtree = S.pop()                              # pop statement tree--------------
        tree1.attach(child1, subtree)                  # attach statment tree to child1
    
    S.append(tree1)
        
        
def stmt(m):
    global S
    global v
    global lookahead
    global tokenval 
    global label_list
    global label_name
    global err_flag
    if lookahead == "CONTINUE": #recognize continue
        match(m, 'CONTINUE')
        subtree = MutableLinkedTree()
        subtree.add_root(tokenval)
        S.append(subtree)
        lookahead = lexan(m)
        match(m, ';')
        lookahead = lexan(m)
        
    elif lookahead == "BREAK": #recognize break
        match(m, 'BREAK')
        subtree = MutableLinkedTree()
        subtree.add_root(tokenval)
        S.append(subtree)
        lookahead = lexan(m)
        match(m, ';')
        lookahead = lexan(m)
        
    elif lookahead == "LABEL": #recognize label
        match(m, 'LABEL')
        subtree = MutableLinkedTree()
        root = subtree.add_root(tokenval)
        lookahead = lexan(m)
        match(m, ':')
        lookahead = lexan(m)
        match(m, 'ID')
        subtree.add_child(root, tokenval)
        S.append(subtree)
        label_name = tokenval #get label name
        label_list = []
        label_list.append(label_name) #append label name to symbol table
        lookahead = lexan(m)
        match(m, ';')
        lookahead = lexan(m)
    elif lookahead == "GOTO": #recognize goto
        match(m, "GOTO")
        subtree = MutableLinkedTree()
        root = subtree.add_root(tokenval)
        lookahead = lexan(m)
        match(m, label_name) #confirm that label exists
        subtree.add_child(root, tokenval)
        S.append(subtree)
        lookahead = lexan(m)
        match(m, ';')
        lookahead = lexan(m)
        
    elif tokenval == "for": #production for (assn ';' cond ; assn){ code }
        match(m, "for")
        subtree = MutableLinkedTree() #create for tree
        root = subtree.add_root(tokenval) #add root node
        lookahead = lexan(m)
        match(m, '(') 
        lookahead = lexan(m)
        assn(m) # call for assignment
        assn1_tree = S.pop() #store 1st assn tree
        cond(m) # check for conditional statement
        con_tree = S.pop() #store conditional tree
        lookahead = lexan(m)
        assn(m) # check for 2nd assignment
        assn2_tree = S.pop() #store 2nd assignment tree
        match(m, ')')
        
        subtree._attacho(root, assn1_tree) #attach assn1
        subtree._attacho(root, con_tree) #attach conditional
        q = subtree.add_child(root, "block")
        subtree._attacho(root, assn2_tree) #attach assn2
        S.append(subtree)
        
        lookahead = lexan(m)
        match(m, '{') #check for opening brace
        lookahead = lexan(m)
        parse(m)
        for_tree = S.pop()
        subtree.attach(q, for_tree)
        match(m, '}')
        S.append(subtree)
        
    elif tokenval == "do": #production for do -> do { code } while (cond);
        match(m, 'do')
        subtree = MutableLinkedTree() #create tree
        root = subtree.add_root(tokenval) #add do node
        lookahead = lexan(m)
        match(m, '{')
        lookahead = lexan(m)
        o = subtree.add_child(root, "open") #confirm do loop is opened, add to tree
        S.append(subtree) #add to stack
        parse(m)
        r = S.pop()
        subtree.attach(o, r)
        match(m, '}')
        lookahead = lexan(m)
        
        match(m, "while")
        w_subtree = MutableLinkedTree()
        w_root = w_subtree.add_root(tokenval) #add while node
        lookahead = lexan(m)
        match(m, '(') # match paran
        lookahead = lexan(m)
        cond(m) #check for conditional statement
        con_tree = S.pop() #store conditional tree
        w_subtree._attacho(w_root, con_tree) #attach conditional tree to root
        subtree._attacho(root, w_subtree)
        S.append(subtree) #add to stack
        match(m, ')')
        lookahead = lexan(m)
        
    elif tokenval == "while":# production for while -> while (cond) { code }
        parse_while(m)
        
    elif tokenval == "if": #production for if (cond) {code} else {code} | if (cond) {code}
        subtree = MutableLinkedTree()
        root = subtree.add_root(tokenval)
        S.append(subtree)
        match(m, 'if')
        lookahead = lexan(m)
        match(m, '(')
        lookahead = lexan(m)
        cond(m) #check for conditional statement
        con_tree = S.pop() #grab tree for conditional statement
        subtree._attacho(root, con_tree) #attach to if statement
        f = subtree.add_child(root, "block")
        match(m, ')')
        lookahead = lexan(m)
        match(m, '{') #check for opening brace
        lookahead = lexan(m)
        parse(m)
        match(m, "}")
        if_tree = S.pop()
        subtree.attach(f, if_tree)
        lookahead = lexan(m)
        root_e = subtree.add_child(root, tokenval)
        match(m, "else") #force else after IF is over
        lookahead = lexan(m)
        match(m, "{") #check for opening brace
        lookahead = lexan(m)
        parse(m)
        match(m, "}")
        e_tree = S.pop()
        subtree._attacho(root_e, e_tree)
        S.append(subtree)
    elif (tokenval == "int"): #recognize INT declarations
        if v: print("parsing a declaration")
        decl(m)
        
    elif (lookahead == "ID"): #recognize ID assignment
        if v: print("parsing an assignment")
        assn(m)
        
    else:
        error("Incorrect Statment Type")

def parse_while(m):
    global lookahead
    global tokenval
    global v
    global S
    global err_flag
    match(m, "while")
    subtree = MutableLinkedTree()
    root = subtree.add_root(tokenval) #add while node
    lookahead = lexan(m)
    match(m, '(') # match paran
    lookahead = lexan(m)
    cond(m) #check for conditional statement
    con_tree = S.pop() #store conditional tree
    subtree._attacho(root, con_tree) #attach conditional tree to root
    q = subtree.add_child(root, "block")
    S.append(subtree) #add to stack
    match(m, ')')
    lookahead = lexan(m)
    match(m, '{') #match opening curly brace
    lookahead = lexan(m)
    parse(m) #call code
    w_tree = S.pop() #pop while tree
    subtree.attach(q, w_tree) #attach code to "block"
    match(m, '}')
    S.append(subtree)
def decl(m):
    #production for decl -> int ID ; | int ID[NUM] ;
    global lookahead
    global tokenval
    global v
    global S
    global err_flag
    subtree = MutableLinkedTree()         #---------------------------------Create sub-tree
    root = subtree.add_root(tokenval)     #---------------------------------Add root node for digit              
    lookahead = lexan(m)
    match(m, "ID")
    dec_id = subtree._add_child(root, lookahead) #add id to subtree
    subtree.add_child(dec_id, tokenval)
    lookahead = lexan(m)
    if lookahead == "PUN":
        if tokenval == ';':
            match(m, ';')
            if vv:printTree(subtree, 1, 2000, 14)
            S.append(subtree) # append final tree
            lookahead = lexan(m)
        elif tokenval == '[':
            match(m, '[')
            lookahead = lexan(m)
            match(m, "NUM")
            dec_num = subtree.add_child(dec_id, tokenval) #append array condition to tree
            lookahead = lexan(m)
            match(m, ']')
            lookahead = lexan(m)
            match(m, ';')
            if vv: printTree(subtree, 1, 2000, 14)
            S.append(subtree) # append final tree
            lookahead = lexan(m)
    else:
        error("error: invalid token on line " + str(line_num-2))

def assn(m):
    #production for assn -> ID = expr | ID[expr] = expr
    global v
    global lookahead
    global S
    global tokenval
    global err_flag
    
    match(m,'ID')       # Match ID
    tree_id = tokenval  #capture id name
    lookahead = lexan(m)# obtain next token
    
    op = tokenval       #capture operator
    if tokenval == '[': # check for array assignment
        if v: print("parsing an array assignment")
        match(m, '[')
        lookahead = lexan(m)
        expr(m)
        arr_expr_tree = S.pop() # Capture Tree for index expression
        match(m, ']')
        lookahead = lexan(m)
        match(m, '=')
        lookahead = lexan(m)
        expr(m)
        expr_tree = S.pop() #store expr
        
        subtree = MutableLinkedTree() #create subtree   #----Not sure about this part yet - Joy
        root = subtree.add_root("asID") #create root
        id_node = subtree.add_child(root, tree_id)
        in_arr = subtree.add_child(root, "array") 
        subtree._attacho(in_arr, arr_expr_tree) #connect array
        subtree._attacho(id_node, expr_tree) #connect expr  #----Not sure about this part yet - Joy
        if vv: printTree(subtree, 1, 2000, 14)
        
        S.append(subtree) #append to stack
        match(m, ';')
        lookahead = lexan(m)
        
    elif tokenval == "=":
        if v: print("parsing a scalar assignment")
        match(m, '=')
        lookahead = lexan(m)
        expr(m)
        expr_tree = S.pop()

        subtree = MutableLinkedTree()
        root = subtree.add_root(op)
        id_node1 = subtree.add_child(root, tree_id)
        subtree.add_child(id_node1, "asID")
        subtree._attacho(root, expr_tree)
        if vv: printTree(subtree, 1, 2000, 14)

        S.append(subtree)
        match(m, ';')
        lookahead = lexan(m)
    else:
        lookahead = DONE         # this will stop the translator when there are errors
        error("error: incorrect OP on line " + str(line_num-2))
        
def cond(m):
    #production -> expr RELOP expr
    global lookahead
    global err_flag
    global tokenval
    global lookahead
    subtree = MutableLinkedTree()
    expr(m)
    expr_tree1 = S.pop()
    match(m, "RELOP")
    root = subtree.add_root(tokenval)
    lookahead = lexan(m)
    expr(m)
    expr_tree2 = S.pop()
    subtree._attacho(root, expr_tree1)
    subtree._attacho(root, expr_tree2)
    S.append(subtree)

def expr(m):
    #production for expr -> term moreterms
    global v
    global S
    global lookahead
    global tokenval
    global err_flag
    if v: print("expression")
    term(m)

    while lookahead == "OP":
        if tokenval == '+' or tokenval == '-':    # while next token is +/-
            match(m,tokenval)
            if v: print("(expr)parsing an additive term")
            op = tokenval
            lookahead = lexan(m)
            
            term(m) # check for next term
            
            if vv:                                         #-------DEBUG TRANSLATION---------
                print("(expr)size of stack:",len(S))
                for T in S:  printTree(T, 0.5, 2000, 14)   #-------DEBUG TRANSLATION---------            
            
            subtree = MutableLinkedTree()         #---------------------------------Create sub-tree
            root = subtree.add_root(op)           #---------------------------------Add root node for digit
            right = S.pop()                       #---------------------------------pop right operand
            left = S.pop()                        #---------------------------------pop left operand
            subtree._attacho(root,left)           #---------------------------------add children to subtree
            subtree._attacho(root,right)
            S.append(subtree)                     #---------------------------------push sub-tree onto stack
            
            continue
        else:
            return

def term(m):
    #term -> factor morefacs and moreterms -> + term | - term | e
    global v
    global S
    global lookahead
    global tokenval
    global err_flag
    if v: print("\t\tterm")
    factor(m) 
    
    while True:
        if tokenval == '*' or tokenval == '/' or tokenval == '%':
            match(m,tokenval)
            if v: print("\t\t(term)parsing a product statment")
            opm = tokenval
            lookahead = lexan(m)
            factor(m)                           
            
            if vv:                                         #-------DEBUG TRANSLATION---------
                print("\t\t(term)size of stack:",len(S))
                for T in S:  printTree(T, 0.5, 2000, 14)   #-------DEBUG TRANSLATION---------
            
            subtree = MutableLinkedTree()         #---------------------------------Create sub-tree
            root = subtree.add_root(opm)          #---------------------------------Add root node for digit
            right = S.pop()                       #---------------------------------pop right operand
            left = S.pop()                        #---------------------------------pop left operand
            subtree._attacho(root,left)           #---------------------------------add children to subtree
            subtree._attacho(root,right)
            S.append(subtree)                     #---------------------------------push sub-tree onto stack
            
            continue
        else:
            return

def factor(m):
    global v
    global S
    global lookahead
    global tokenval
    global err_flag
    if v: print("\t\t\tfactor")
    
    if tokenval=='(':
        if v: print("\t\t\t(factor)parsing parenthetic expression")
        match(m,'(')
        lookahead = lexan(m)
        expr(m)       # recursively check for more expressions
        match(m,'(')
        lookahead = lexan(m)
        
    elif lookahead=="ID":
        name = lookahead
        
        match(m,lookahead)
        if v: print("\t\t\tparsing an ID or NUM factor")
        subtree = MutableLinkedTree()         #---------------------------------Create sub-tree
        root = subtree.add_root(tokenval)            #---------------------------------Add root node for digit
        lookahead = lexan(m)
        
        if tokenval == '[':                   #---------------------------------check for array on right hand side
            match(m, '[')
            lookahead = lexan(m)
            
            expr(m)
            expr_tree = S.pop()
            subtree._attacho(root, expr_tree)
            
            match(m,']')
            lookahead = lexan(m)
        
        S.append(subtree)                     #---------------------------------push onto stack
        
    
    elif lookahead == "NUM":
        match(m,lookahead)
        if v: print("\t\t\tparsing an ID or NUM factor")
        subtree = MutableLinkedTree()         #---------------------------------Create sub-tree
        subtree.add_root(tokenval)            #---------------------------------Add root node for digit
        S.append(subtree)                     #---------------------------------push onto stack
        lookahead = lexan(m)
    
    elif tokenval == None:
        return
    else:
        return


In [51]:
def match(m,t):
    """Checks for token t and then calls the lexical analyzer for the next token"""
    global lookahead
    global tokenval
    global v
    global err_flag
    if tokenval == t or lookahead == t:                         # check that the tokens match
        if v: print("MATCH! " + str(tokenval))
    else:
        error("syntax error: character ("+str(tokenval)+") does not match ("+str(t)+") on line " + str(line_num))# error message

def error(err_msg):
    """Produces the output for error messages"""
    global line_num     # current line number
    global err_flag
    global lookahead    # current token
    global DONE         # FLAG for end of iteration
    
    print("line "+str(line_num)+": "+err_msg)         # print error message
    err_flag = True
    
def emit(t,tokenval):
    """Produces the output of the parser"""
    
    if t in ["+","-","*","/"]:                        # arithmetic ops
        print(t)
    elif t=="NUM":                                    # constants
        print(tokenval)
    elif t=="ID":                                     # variables
        print(ST[tokenval][0])
    else:
        print("token =",t,", tokenval =",tokenval)    # unkown(?) tokens     

In [1]:
def translate(m):
    """converts the input string from infix to postix notation"""
    
    init()                   # the translator needs to call init() for your lexical analyzer
    global err_flag
    global S                 # Stack
    global v
    global vv
    
    v = False
    vv = False
    
    S = []
    
    parse(m)                 # Call parse function to begin translation
    
    T = S.pop()
    
    return T